In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-14 05:23:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-12-14 05:23:14 (6.57 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
# Start a Spark session with the additional option that adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in video games dataset from S3 Buckets
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Count the number of records in the Video Games DataFrame
video_games_df.count()

1785997

In [ ]:
# Drop null values
dropna_df = video_games_df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [ ]:
# Load in a sql function to us to_date
from pyspark.sql.functions import to_date


In [ ]:
# Create the review_id DataFrame
review_id_df = clean_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10010D6B1QM7U|   26197704|B004HX5OFW|       8155204| 2013-03-28|
|R100BI61LIKH2N|   10409381|B000QW9D14|     583724133| 2013-10-28|
|R100TPJCYDYGBU|   39071377|B00D3IKP7Y|     378642975| 2013-12-31|
|R1018OO9U6UUWO|   32826337|B0094X28J0|     272196441| 2014-03-21|
|R102YIWZEIAXT9|   16385120|B000UQAUWW|     694091073| 2007-11-29|
|R1034S7FA21OI1|   14198390|B004S5PBM0|     275810674| 2011-11-06|
|R103LC8CTAHWZ6|   16965219|B002EE7OKE|     852505908| 2013-10-07|
|R103PT7S4HRIP8|   48295356|B003R7H5TC|     318359987| 2010-12-28|
|R104RCQZYPYDXS|   21497984|B000R3BNDI|     318965716| 2008-02-28|
|R105KZIXHZW63W|   22595160|B000NKFEZI|     256274221| 2013-02-04|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the products DataFrame and drop the duplicates
products_df = clean_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00O9GP9AM|Little Mac amiibo...|
|B000AOEU2K|Fire Emblem: Path...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B00MUTAVH6|Under Night In-Bi...|
|B000IMYKQ0|Wii Nunchuk Contr...|
|B00004S3AI|Populous:  The Be...|
|B00YT90JWC|Red Wii Mini Cons...|
|B00DHF39KS|Wolfenstein: The ...|
|B00004TJ2N|Zeus: Master of O...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Create the customers DataFrame
customers_df = clean_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   40504830|             8|
|    3675423|             3|
|   11393715|             9|
|   14175902|            22|
|   39664150|             4|
|   14534785|             1|
|   47765241|             7|
|   33929721|             1|
|   41446882|             8|
|   52714119|             4|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create the vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10010D6B1QM7U|          1|            0|          0|   N|
|R100BI61LIKH2N|          5|            1|          2|   N|
|R100TPJCYDYGBU|          3|            1|          2|   N|
|R1018OO9U6UUWO|          5|            0|          0|   N|
|R102YIWZEIAXT9|          5|            2|          4|   N|
|R1034S7FA21OI1|          5|            2|          2|   N|
|R103LC8CTAHWZ6|          2|            1|          1|   N|
|R103PT7S4HRIP8|          2|            0|          0|   N|
|R104RCQZYPYDXS|          5|            7|          8|   N|
|R105KZIXHZW63W|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

Enter database password··········


In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazonchallenge.czbxze2tryse.us-east-2.rds.amazonaws.com:5432/amazonchallenge"
config = {"user":"postgres",
          "password": "Orquidea1%",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataFrame to review_id table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [ ]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)